In [5]:
import random
import json
from datetime import datetime

def simulate_sensors():
    data = {
        "timestamp": datetime.now().isoformat(),
        "soil_moisture": random.uniform(10, 80),
        "temperature": random.uniform(15, 40),
        "humidity": random.uniform(20, 90),
        "light": random.uniform(0, 1000)
    }
    print(json.dumps(data, indent=2))

simulate_sensors()

{
  "timestamp": "2025-07-06T17:18:44.698587",
  "soil_moisture": 45.95321544684093,
  "temperature": 27.99105985276013,
  "humidity": 62.37306572846317,
  "light": 362.59666925640136
}


In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.ensemble import RandomForestRegressor

# Load your dataset (replace with your data)
# df = pd.read_csv("/content/your_data.csv")
X_train = np.random.rand(100, 4)
y_train = np.random.rand(100)

# LSTM Model
lstm_model = Sequential([
    LSTM(64, input_shape=(1, 4)),  # 1 timestep, 4 features
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X_train.reshape(-1, 1, 4), y_train, epochs=5)

# Random Forest
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

print("Models trained!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.3264
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2754 
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2562 
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2253 
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1902 
Models trained!


In [7]:
from flask import Flask, jsonify
import numpy as np
from google.colab.output import eval_js
import threading
import socket

app = Flask(__name__)

# Function to find available port
def find_free_port():
    for port in range(6006, 9000):
        with socket.socket() as s:
            if s.connect_ex(('localhost', port)) != 0:
                return port
    raise ValueError("No free ports")

# Add root route
@app.route('/')
def home():
    return "Go to <a href='/predict'>/predict</a> for yield predictions"

# Prediction endpoint
@app.route('/predict')
def predict():
    mock_data = np.random.rand(1, 4)  # Replace with real data
    lstm_pred = lstm_model.predict(mock_data.reshape(1, 1, 4))
    return jsonify({"predicted_yield": float(lstm_pred[0][0])})

# Start server
port = find_free_port()
threading.Thread(
    target=app.run,
    kwargs={'host':'0.0.0.0', 'port':port},
    daemon=True
).start()

# Get URL
print(" * Access endpoints:")
print(f" * Home: {eval_js(f'google.colab.kernel.proxyPort({port})')}")
print(f" * Predict: {eval_js(f'google.colab.kernel.proxyPort({port})')}/predict")

 * Serving Flask app '__main__'
 * Access endpoints:
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6007
 * Running on http://172.28.0.12:6007
INFO:werkzeug:Press CTRL+C to quit


 * Home: https://6007-gpu-t4-s-22t7h2kpof164-a.us-west4-0.prod.colab.dev
 * Predict: https://6007-gpu-t4-s-22t7h2kpof164-a.us-west4-0.prod.colab.dev/predict
